In [52]:
import os
import pickle
import tsplib95
import csv
import time
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import *

In [53]:
np.random.seed(49)

In [54]:
def get_tsp_graph(name) -> nx.Graph:
    # load from pkl file if exists
    graph_file_path = os.path.join("tsp_dataset", f"{name}.graph.pkl")
    if os.path.exists(graph_file_path):
        # print(f"Loading {name} from pkl {graph_file_path}")
        with open(graph_file_path, "rb") as file:
            G = pickle.load(file)
        return G.to_undirected()

    tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
    # print(f"Loading {name} from tsplib {tsp_file_path}")
    problem = tsplib95.load(tsp_file_path)
    G = problem.get_graph()

    # remove redundant edges
    if problem.edge_weight_type == "EXPLICIT" and problem.edge_weight_format == "FULL_MATRIX":
        for u in G.nodes:
            for v in G.nodes:
                if u > v:
                    G.remove_edge(u, v)

    # remove self loops
    loop_edges = list(nx.selfloop_edges(G))
    G.remove_edges_from(loop_edges)

    # save as pkl file
    with open(graph_file_path, "wb") as file:
        print(f"Saving {name} to pkl {graph_file_path}")
        pickle.dump(G, file)

    return G

In [55]:
tsp_dataset_file_path = os.path.join("tsp_dataset", "tsp_dataset.csv")
print(f"Loading tsp dataset from {tsp_dataset_file_path}")
tsp_dataset = pd.read_csv(tsp_dataset_file_path)

Loading tsp dataset from tsp_dataset\tsp_dataset.csv


In [56]:
def get_opt_tour_length(name):
    return tsp_dataset[tsp_dataset["name"] == name]["opt_tour_length"].values[0]

In [57]:
def get_tour_length(G: nx.graph, tour: list):
    tour_length = 0
    for i in range(len(tour) - 1):
        u, v = tour[i], tour[i + 1]
        w = G[u][v]["weight"]
        tour_length += w
    return tour_length

In [58]:
def get_apx_ratio(G: nx.graph, tour: list):
    return get_tour_length(G, tour) / get_opt_tour_length(G.name)

In [59]:
tsp_dataset

,name,dimension,opt_tour_length,type,comment,edge_weight_type,edge_weight_format
0,burma14,14,3323,TSP,14-Staedte in Burma (Zaw Win),GEO,FUNCTION
1,ulysses16,16,6859,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,NaN
2,gr17,17,2085,TSP,17-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
3,gr21,21,2707,TSP,21-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
4,ulysses22,22,7013,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,NaN
...,...,...,...,...,...,...,...
91,u2319,2319,234256,TSP,Drilling problem (Reinelt),EUC_2D,NaN
92,pr2392,2392,378032,TSP,2392-city problem (Padberg/Rinaldi),EUC_2D,NaN
93,pcb3038,3038,137694,TSP,Drilling problem (Junger/Reinelt),EUC_2D,NaN
94,fnl4461,4461,182566,TSP,Die 5 neuen Laender Deutschlands (Ex-DDR) (Bac...,EUC_2D,NaN


In [60]:
from mst import MST
st = MST()

In [61]:
def get_total_edge_weight(G: nx.graph):
    total_edge_weight = 0
    for u, v in G.edges:
        total_edge_weight += G[u][v]["weight"]
    return total_edge_weight

In [62]:
from simulated_annealing import SimulateAnnealing

In [63]:
def get_rand_tree(G: nx.graph):
    T = nx.Graph()
    nodes = list(G.nodes)
    np.random.shuffle(nodes)
    for i in range(len(nodes) - 1):
        u, v = nodes[i], nodes[i + 1]
        T.add_edge(u, v, weight=G[u][v]["weight"])
    return T

In [65]:
with open("rand_results.csv", "a", newline="") as csvfile:
    writer = csv.writer(csvfile)

    sample_list = [
        "fri26",
        "bays29",
        "swiss42",
        "hk48",
        "brazil58",
        "st70",
        "kroB150",
        "pr152",
        "rat195",
        "d657",
    ]
    for name in tsp_dataset["name"]:
        if name not in sample_list:
            continue
        print(name)

        G = get_tsp_graph(name)
        dimension = G.number_of_nodes()
        opt_tour_length = get_opt_tour_length(name)

        d = {
            "name": name,
            "dimension": dimension,
            "opt_tour_length": opt_tour_length,
        }

        tree = st.get_mst_k(G, k)
        tour = christofides(G, tree=tree)
        tour_length = get_tour_length(G, tour)

        c_apx_ratio = get_apx_ratio(G, tour)
        c_tour_length = tour_length

        tree = get_rand_tree(G)
        tour = christofides(G, tree=tree)
        tour_length = get_tour_length(G, tour)

        r_apx_ratio = get_apx_ratio(G, tour)
        r_tour_length = tour_length

        d["c_tour_length"] = c_tour_length
        d["r_tour_length"] = r_tour_length
        d["c_apx_ratio"] = c_apx_ratio
        d["r_apx_ratio"] = r_apx_ratio

        print(list(d.values()))
        writer.writerow(d.values())
        csvfile.flush()

        print()

fri26
['fri26', 26, 937, 988, 2986, 1.0544290288153682, 3.1867662753468515]

bays29
['bays29', 29, 2020, 2188, 6465, 1.0831683168316832, 3.2004950495049505]

swiss42
['swiss42', 42, 1273, 1399, 4870, 1.09897879025923, 3.82560879811469]

hk48
['hk48', 48, 11461, 13059, 55322, 1.1394293691649944, 4.82697844865195]

brazil58
['brazil58', 58, 25395, 27495, 123671, 1.082693443591258, 4.869895648749754]

st70
['st70', 70, 675, 768, 3561, 1.1377777777777778, 5.275555555555556]

kroB150
['kroB150', 150, 26130, 29669, 255216, 1.135438193647149, 9.767164179104478]

pr152
['pr152', 152, 73682, 79317, 1040585, 1.0764772943188297, 14.122648679460383]

rat195
['rat195', 195, 2323, 2704, 20810, 1.164012053379251, 8.958243650452001]

d657
['d657', 657, 48912, 54891, 847503, 1.1222399411187438, 17.327097644749756]



In [68]:
df = pd.read_csv("rand_results.csv")

In [69]:
df

,name,dimension,opt_tour_length,c_tour_length,r_tour_length,c_apx_ratio,r_apx_ratio
0,fri26,26,937,988,2986,1.054429,3.186766
1,bays29,29,2020,2188,6465,1.083168,3.200495
2,swiss42,42,1273,1399,4870,1.098979,3.825609
3,hk48,48,11461,13059,55322,1.139429,4.826978
4,brazil58,58,25395,27495,123671,1.082693,4.869896
5,st70,70,675,768,3561,1.137778,5.275556
6,kroB150,150,26130,29669,255216,1.135438,9.767164
7,pr152,152,73682,79317,1040585,1.076477,14.122649
8,rat195,195,2323,2704,20810,1.164012,8.958244
9,d657,657,48912,54891,847503,1.122240,17.327098


In [70]:
# keep only c_tour_length, r_tour_length
df = df[["name", "dimension", "c_apx_ratio", "r_apx_ratio"]]

In [71]:
df

,name,dimension,c_apx_ratio,r_apx_ratio
0,fri26,26,1.054429,3.186766
1,bays29,29,1.083168,3.200495
2,swiss42,42,1.098979,3.825609
3,hk48,48,1.139429,4.826978
4,brazil58,58,1.082693,4.869896
5,st70,70,1.137778,5.275556
6,kroB150,150,1.135438,9.767164
7,pr152,152,1.076477,14.122649
8,rat195,195,1.164012,8.958244
9,d657,657,1.122240,17.327098


In [ ]:
df.round(6)
df.to_csv("r_results.csv", index=False)